<a href="https://colab.research.google.com/github/ragboom7/EPL-matches-predictor/blob/main/Webscrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd

In [ ]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [ ]:
data = requests.get(standings_url)

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]

In [ ]:
team_urls = [f"https://fbref.com{l}" for l in links]
data = requests.get(team_urls[0])

In [ ]:
matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

In [ ]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [ ]:
data = requests.get(f"https://fbref.com{links[0]}")
shooting = pd.read_html(data.text, match="Shooting")[0]

In [ ]:
shooting.columns = shooting.columns.droplevel()
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

In [ ]:
years = list(range(2024, 2017, -1))
bro = []

In [ ]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [ ]:
import time
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]

    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    for team_url in team_urls:
      team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
      data = requests.get(team_url)
      matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
      soup = BeautifulSoup(data.text)
      links = [l.get("href") for l in soup.find_all('a')]
      links = [l for l in links if l and 'all_comps/shooting/' in l]
      data = requests.get(f"https://fbref.com{links[0]}")
      shooting = pd.read_html(data.text, match="Shooting")[0]
      shooting.columns = shooting.columns.droplevel()
      team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt","SoT%","G/Sh","G/SoT"]], on="Date")
      links = [l.get("href") for l in soup.find_all('a')]
      links = [l for l in links if l and 'all_comps/keeper/' in l]
      data = requests.get(f"https://fbref.com{links[0]}")
      keeping = pd.read_html(data.text, match="Goalkeeping")[0]
      keeping.columns = keeping.columns.droplevel()
      team_data1 = team_data.merge(keeping[["Date", "SoTA", "Saves", "Save%","CS","PSxG"]], on="Date")
      links = [l.get("href") for l in soup.find_all('a')]
      links = [l for l in links if l and 'all_comps/passing/' in l]
      data = requests.get(f"https://fbref.com{links[0]}")
      passing = pd.read_html(data.text, match="Passing")[0]
      passing.columns = passing.columns.droplevel()
      team_data2 = team_data1.merge(passing[["Date", "Cmp", "Att", "Cmp%","TotDist","PrgDist","KP","PPA","PrgP"]], on="Date")
      links = [l.get("href") for l in soup.find_all('a')]
      links = [l for l in links if l and 'all_comps/defense/' in l]
      data = requests.get(f"https://fbref.com{links[0]}")
      defense = pd.read_html(data.text, match="Defensive Actions")[0]
      defense.columns = defense.columns.droplevel()
      team_data3 = team_data2.merge(defense[["Date", "Tkl", "TklW", "Blocks","Sh","Pass","Int","Err","Tkl+Int"]], on="Date")
      links = [l.get("href") for l in soup.find_all('a')]
      links = [l for l in links if l and 'all_comps/possession/' in l]
      data = requests.get(f"https://fbref.com{links[0]}")
      poss = pd.read_html(data.text, match="Possession")[0]
      poss.columns = poss.columns.droplevel()
      team_data4 = team_data3.merge(poss[["Date", "Touches", "Att Pen", "Att","Carries","PrgDist","CPA","Rec","PrgR"]], on="Date")
      links = [l.get("href") for l in soup.find_all('a')]
      links = [l for l in links if l and 'all_comps/misc/' in l]
      data = requests.get(f"https://fbref.com{links[0]}")
      misc = pd.read_html(data.text, match="Miscellaneous Stats")[0]
      misc.columns = misc.columns.droplevel()
      team_data5 = team_data4.merge(misc[["Date", "Fls","Fld","Crs"]], on="Date")
      team_data = team_data5[team_data5["Comp"] == "Premier League"]
      team_data.loc[team_data['Date'].str.startswith('2025'), 'Season'] = 2025
      team_data.loc[team_data['Date'].str.startswith('2024'), 'Season'] = 2024
      team_data['Team']=team_name
      bro.append(team_data)
      time.sleep(1)

'\nimport time\nfor year in years:\n    data = requests.get(standings_url)\n    soup = BeautifulSoup(data.text)\n    standings_table = soup.select(\'table.stats_table\')[0]\n\n    links = [l.get("href") for l in standings_table.find_all(\'a\')]\n    links = [l for l in links if \'/squads/\' in l]\n    team_urls = [f"https://fbref.com{l}" for l in links]\n    \n    previous_season = soup.select("a.prev")[0].get("href")\n    standings_url = f"https://fbref.com{previous_season}"\n    for team_url in team_urls:\n      team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")\n      data = requests.get(team_url)\n      matches = pd.read_html(data.text, match="Scores & Fixtures")[0]\n      soup = BeautifulSoup(data.text)\n      links = [l.get("href") for l in soup.find_all(\'a\')]\n      links = [l for l in links if l and \'all_comps/shooting/\' in l]\n      data = requests.get(f"https://fbref.com{links[0]}")\n      shooting = pd.read_html(data.text, match="Shooting")[0]\n 

In [ ]:
match_df1 = pd.concat(bro)
match_df1.columns = [c.lower() for c in match_df1.columns]

In [ ]:
match_df1.to_csv(r"C:\Users\ragha\Downloads\eplfinal.csv", index=False)
from google.colab import files
files.download(r"C:\Users\ragha\Downloads\eplfinal.csv")